In [74]:
import numpy as np
import sympy as sp
sp.init_printing()

In [75]:
def calculate_chrst_sym_4d(metric, inv_metric):
  #Calculate the christoffel symbols for a given metric

  #instantiate variables
  t0, t1, t2, t3 = sp.symbols('t0 t1 t2 t3') 


  chrst = np.zeros((4, 4, 4), dtype = type(sp.sin(t3))) #empty array to put our christoffel symbols

  for index, val in np.ndenumerate(chrst):

    mod_chrst = 0

    #making indicies the same as our computational formula

    k = index[0]
    i = index[1]
    j = index[2]


    for m in range(4):
      mod_chrst = mod_chrst + .5 * inv_metric[k][m] * (
          sp.diff(metric[m][i], 't' + str(j)) + sp.diff(metric[m][j], 't' + str(i)) - sp.diff(metric[i][j], 't' + str(m))
      )


    chrst[k][i][j] = mod_chrst

  return chrst

In [76]:
from sympy.stats import E

def sym_cub_tensor(der):
    #symmetric cubic tensor given n derivatives
    
    n = len(der)    
    return_mat = np.zeros((n,n,n), dtype = 'object')
    
    for k in range(n):
        for i in range(n):
            for j in range(n):
                return_mat[k][i][j] = sp.simplify(E(der[k] * der[i] * der[j]))
    
    return return_mat

In [93]:
def find_alpha_chrst(chrst, tijks, alpha):
    #finds alpha christoffel symbols
    return chrst + alpha * tijks

In [78]:
def alpha_curvature_tensor(alpha_chrsts):
    #finds standard curvature
    
    varis = [t0,t1,t2,t3]
    
    alpha_curve_tensor_arr = np.zeros((4,4,4,4), dtype = 'object')
    
    for l in range(4):
        for i in range(4):
            for j in range(4):
                for k in range(4):
                    pre_sum = sp.diff(alpha_chrsts[l][i][k], varis[j]) - sp.diff(alpha_chrsts[l][j][k], varis[i])
                    for p in range(4):
                        pre_sum += alpha_chrsts[p][i][k] * alpha_chrsts[l][j][p] \
                        - alpha_chrsts[p][j][k] * alpha_chrsts[l][i][p]
                    alpha_curve_tensor_arr[l][i][j][k] = sp.simplify(pre_sum)
    
    return alpha_curve_tensor_arr

In [79]:
def curvature_components(curv_tens, metric):
    #finds curvature components
    
    n = np.shape(curv_tens)[0]
    
    return_arr = np.zeros((n,n,n,n), dtype = 'object')
    
    for index, val in np.ndenumerate(return_arr):
        pre_sum = 0
        for s in range(n):
            pre_sum += curv_tens[s][index[1]][index[2]][index[3]] * metric[index[3]][s]
        return_arr[index[0]][index[1]][index[2]][index[3]] = pre_sum
        
    return return_arr

In [65]:
#creating variables for use

t0 = sp.Symbol('t0', real=True)
t1 = sp.Symbol('t1', positive = True)
t2 = sp.Symbol('t2', real=True)
t3 = sp.Symbol('t3', positive = True)

from sympy.stats import Normal

x, y = Normal('x', t0, t1), Normal('y', t2, t3)

In [80]:
#S2 x S2 metric from bad papper

s22_met = np.zeros((4,4), dtype = 'object')
s22_met[0][0] = 1
s22_met[1][1] = sp.sin(t0) ** 2
s22_met[2][2] = 1
s22_met[3][3] = sp.sin(t2) ** 2

In [81]:
#S2 x S2 inverse metric from bad papper

s22_inv_met = np.zeros((4,4), dtype = 'object')
s22_inv_met[0][0] = 1
s22_inv_met[1][1] = sp.sin(t0) ** (-2)
s22_inv_met[2][2] = 1
s22_inv_met[3][3] = sp.sin(t2) ** (-2)

In [82]:
#H x H metric

h22_met = np.zeros((4,4), dtype = 'object')
h22_met[0][0] = 1
h22_met[1][1] = 1 / t1
h22_met[2][2] = 1
h22_met[3][3] = 1 / t3

In [86]:
#H x H inverse metric

h22_inv_met = np.zeros((4,4), dtype = 'object')
h22_inv_met[0][0] = 1
h22_inv_met[1][1] = t1
h22_inv_met[2][2] = 1
h22_inv_met[3][3] = t3

In [87]:
#We perform step 3
step_3_s22 = calculate_chrst_sym_4d(s22_met, s22_inv_met)

step_3_h22 = calculate_chrst_sym_4d(h22_met, h22_inv_met)

In [88]:
#For step 4 we first define the derivatives

der_1 = (x - t0) / (t1 ** 2)
der_2 = (((x - t0) ** 2) / (t1 ** 3)) - 1 / t1
#derivative take wrt variance
#der_2 = (((x - t0) ** 2) / (2 * (t1 ** 4))) - 1 /(2 * t1 ** 2)
der_3 = (y - t2) / (t3 ** 2)
der_4 = (((y - t2) ** 2) / (t3 ** 3)) - 1 / t3
#der_4 = (((y - t2) ** 2) / (2 * (t3 ** 4))) - 1 /(2 * t3 ** 2)

der_list = [der_1,der_2,der_3,der_4]

In [91]:
#Step 4 calculation
#try to not run this cell multiple times
step_4_tensor = sym_cub_tensor(der_list)

In [94]:
step_5_arr_pos1 = find_alpha_chrst(step_3_h22, step_4_tensor, 1)
step_5_arr_neg1 = find_alpha_chrst(step_3_h22, step_4_tensor, -1)

In [96]:
step_6_arr_pos1 = alpha_curvature_tensor(step_5_arr_pos1)
step_6_arr_neg1 = alpha_curvature_tensor(step_5_arr_neg1)

In [58]:
step_7_arr = curvature_components(step_6_arr, new_s_met)